In [1]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from torch.utils.data import DataLoader

In [2]:
# Verificar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [3]:
# Configuración de parámetros
model_name = "Rostlab/prot_xlnet"
batch_size = 100
num_epochs = 3
learning_rate = 5e-5
max_length = 512
output_dir = "../models/prot_xlnet_finetuned"

In [4]:
# Cargar tokenizador y modelo preentrenado
print("Cargando tokenizador y modelo...")
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Ajustar según tu tarea
model.to(device)

Cargando tokenizador y modelo...


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_xlnet and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(37, 1024)
    (layer): ModuleList(
      (0-29): 30 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): ReLU()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=1024, out_features=1024, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last_dropo

In [5]:
# Para congelar las capas base y solo entrenar las nuevas
for param in model.transformer.parameters():
    param.requires_grad = False

In [6]:
# Ejemplo de datos (reemplazar con tus propios datos)
# Para este ejemplo, usaré un conjunto de datos simulado de secuencias de proteínas y sus etiquetas
def create_dummy_data():
    # Secuencias de ejemplo (estas deberían ser reemplazadas con secuencias reales)
    sequences = [
        "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",
        "MASNTVSAQGGSNRPVRDLASRQDFVRASSIIEKQLRDKVSADDLPVTLAQHLAVNFLHVLRLLE",
        "MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSS",
        # ... más secuencias
    ]
    # Etiquetas simuladas: 0 o 1 (por ejemplo, para una tarea de clasificación binaria)
    labels = [0, 1, 0]
    
    # Crear más datos simulados
    for i in range(100):
        amino_acids = "ACDEFGHIKLMNPQRSTVWY"
        seq_length = np.random.randint(50, 200)
        seq = ''.join(np.random.choice(list(amino_acids)) for _ in range(seq_length))
        sequences.append(seq)
        labels.append(np.random.randint(0, 2))
    
    return pd.DataFrame({"sequence": sequences, "label": labels})


In [7]:
# Cargar datos
# En un caso real, cargarías tus propios datos así:
#df = pd.read_csv("../data/raw/peptides_labeled.csv")
df = create_dummy_data()

In [8]:
# Dividir en conjuntos de entrenamiento y validación
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
# Función para tokenizar las secuencias
def tokenize_function(examples):
    return tokenizer(
        examples["sequence"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

In [10]:
# Convertir DataFrames a datasets de HuggingFace
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenizar los datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Reformatear para Trainer
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [11]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=learning_rate,
)

c:\TMP_CASES\DEV\GIT\ADVANCE_DRUG_DISCOVERY\ADVANCED_DRUG_DISCOVERY\.VENV\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Definir la función de cálculo de métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}


In [13]:
# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
# Entrenar el modelo
print("Iniciando entrenamiento...")
trainer.train()


Iniciando entrenamiento...


KeyboardInterrupt: 

In [ ]:
# Guardar el modelo y el tokenizador
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Modelo guardado en {output_dir}")

In [ ]:
# Evaluar el modelo en el conjunto de validación
print("Evaluando el modelo...")
eval_results = trainer.evaluate()
print(f"Resultados de evaluación: {eval_results}")

In [ ]:
# Ejemplo de cómo usar el modelo para hacer predicciones
def predict_sequence(sequence):
    inputs = tokenizer(
        sequence,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
    
    return predicted_class, predictions[0].tolist()

In [ ]:
# Probar con una secuencia de ejemplo
test_sequence = "MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSS"
predicted_class, class_probs = predict_sequence(test_sequence)
print(f"Clase predicha: {predicted_class}")
print(f"Probabilidades de clase: {class_probs}")


In [ ]:
# Uso avanzado: Fine-tuning con diferentes tasas de aprendizaje por capas
# Esto es útil para reentrenar selectivamente ciertas partes del modelo

# Ejemplo de cómo configurar diferentes tasas de aprendizaje para diferentes capas
def setup_optimizer_with_layerwise_lr(model, base_lr=5e-5, decay_factor=0.9):
    # Agrupar parámetros por profundidad en la red
    # Las capas más profundas tienen tasas de aprendizaje más altas
    layer_params = []
    
    # Parámetros del embedding con la tasa de aprendizaje más baja
    layer_params.append({
        'params': model.transformer.word_embedding.parameters(),
        'lr': base_lr * (decay_factor ** 12)
    })
    
    # Parámetros de las capas de transformers con tasas de aprendizaje graduales
    for i, layer in enumerate(model.transformer.layer):
        layer_params.append({
            'params': layer.parameters(),
            'lr': base_lr * (decay_factor ** (12 - i))
        })
    
    # Parámetros de la capa de clasificación con la tasa de aprendizaje más alta
    layer_params.append({
        'params': model.sequence_summary.parameters(),
        'lr': base_lr
    })
    layer_params.append({
        'params': model.logits_proj.parameters(),
        'lr': base_lr
    })
    
    # Crear el optimizador con los grupos de parámetros
    optimizer = torch.optim.AdamW(layer_params)
    return optimizer

# Comentado porque usamos Trainer en este ejemplo, pero puedes descomentar y modificar
# para entrenamiento manual con tasas de aprendizaje personalizadas
"""
# Configurar optimizer con tasas de aprendizaje por capas
optimizer = setup_optimizer_with_layerwise_lr(model)

# Configurar scheduler para reducir tasa de aprendizaje durante el entrenamiento
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Loop de entrenamiento manual
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Mover batch a device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    # Actualizar tasa de aprendizaje
    scheduler.step()
"""